In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_csv("/content/drive/MyDrive/articles_rows.csv")

In [ ]:
df.head()

,id,title,summary,original_url,source_id,published_at,content_text,language_code,image_url,sentiment,actors,place,topic,subtopic,created_at,updated_at
0,1144af4f-9e0c-4673-9c45-ae02336d69ca,Mappls vs Google Maps: 5 Features that make In...,"Mappls, a homegrown navigation app in India, h...",https://www.indiatvnews.com/technology/news/ma...,65f7d197-b7d8-4051-a230-0536cf4816fc,2025-10-20 06:21:00+00,NaN,en-IN,https://resize.indiatvnews.com/en/resize/newbu...,0.8,"[""Mappls"",""Google Maps"",""India""]",India,Technology,Software/Apps,2025-10-20 07:49:28.73+00,2025-10-20 07:49:28.731+00
1,40854a5a-15a1-4153-a1ea-4b6eaab657f7,India Technology News: Absolute Security Annou...,"Absolute Security, a global provider of cybers...",https://www.businesswireindia.com/india-techno...,367ad881-45e5-4381-b473-955fb713fffa,2025-09-23 07:00:00+00,NaN,en-IN,NaN,0.8,"[""Absolute Security"",""Absolute Software"",""India""]",India,Business,Technology and IT,2025-10-20 07:49:29.28+00,2025-10-20 07:49:29.28+00
2,a149ac95-d2c4-4907-9afe-d9aae91020fd,"India Develops First Indigenous Antibiotic, Na...",India has developed its first indigenous antib...,https://www.newsonair.gov.in/india-develops-fi...,c7486ee3-c0b9-4bdf-b5e3-ab860abd3946,2025-10-19 02:08:00+00,NaN,en-IN,https://www.newsonair.gov.in/wp-content/upload...,0.8,"[""Dr Jitendra Singh"",""Union Science and Techno...",India,Science,Health,2025-10-20 07:49:28.884+00,2025-10-20 07:49:28.884+00
3,d5f2fbd6-62b0-4d8d-9ee5-5f3adfa0533e,Taiwan seeks bigger tech presence in India as ...,Taiwan is looking to increase its presence in ...,https://www.thehindu.com/sci-tech/technology/t...,7436dab8-b98b-4394-99e8-bac420786c23,2025-09-27 07:00:00+00,NaN,en-IN,https://th-i.thgim.com/public/incoming/p9zefe/...,0.5,"[""Taiwan"",""India"",""US"",""Taiwan External Trade ...",India,Business,Technology and Trade,2025-10-20 07:49:29.147+00,2025-10-20 07:49:29.147+00
4,d6e24caf-ce90-4243-8833-332f923d1058,Gemini Nano Banana trend: 5 AI prompts to crea...,The article discusses the 'Gemini Nano Banana ...,https://timesofindia.indiatimes.com/technology...,225efd35-15e1-47e4-a8ed-fe7cea927c8f,2025-10-20 03:42:00+00,NaN,en-IN,https://static.toiimg.com/thumb/msid-124698899...,0.8,"[""Gemini Nano"",""Diwali"",""AI""]",India,Technology,Artificial Intelligence,2025-10-20 07:49:29.017+00,2025-10-20 07:49:29.017+00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5 non-null      object 
 1   title          5 non-null      object 
 2   summary        5 non-null      object 
 3   original_url   5 non-null      object 
 4   source_id      5 non-null      object 
 5   published_at   5 non-null      object 
 6   content_text   0 non-null      float64
 7   language_code  5 non-null      object 
 8   image_url      4 non-null      object 
 9   sentiment      5 non-null      float64
 10  actors         5 non-null      object 
 11  place          5 non-null      object 
 12  topic          5 non-null      object 
 13  subtopic       5 non-null      object 
 14  created_at     5 non-null      object 
 15  updated_at     5 non-null      object 
dtypes: float64(2), object(14)
memory usage: 772.0+ bytes


In [ ]:
tfv=TfidfVectorizer(min_df=3,max_features=None,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3),stop_words='english')
df['summary']=df['summary'].fillna('')

In [ ]:
tfv_matrix=tfv.fit_transform(df['summary'])

In [ ]:
tfv_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (5, 3)>

In [ ]:
tfv_matrix.shape

(5, 3)

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [ ]:
sig[0]

array([0.87006166, 0.83378034, 0.82440373, 0.81991783, 0.76159416])

In [ ]:
indices=pd.Series(df.index,index=df['title']).drop_duplicates()

In [ ]:
indices

,0
title,
Mappls vs Google Maps: 5 Features that make India’s homegrown navigation app smarter for Indian roads,0
India Technology News: Absolute Security Announces Plans to Expand Presence in India,1
"India Develops First Indigenous Antibiotic, Nafithromycin, Says Union Science and Tech Min Dr Jitendra Singh",2
"Taiwan seeks bigger tech presence in India as U.S. demand rises, trade body chief says",3
Gemini Nano Banana trend: 5 AI prompts to create stunning Diwali couple portraits,4


In [ ]:
def give_rec(title,sig=sig):
  idx=indices[title]
  sig_scores=list(enumerate(sig[idx]))
  sig_scores=sorted(sig_scores,key=lambda x:x[1],reverse=True)
  sig_scores=sig_scores[1:11]
  article_indices=[i[0] for i in sig_scores]
  return df['title'].iloc[article_indices]

In [ ]:
give_rec('India Technology News: Absolute Security Announces Plans to Expand Presence in India')

,title
3,Taiwan seeks bigger tech presence in India as ...
2,"India Develops First Indigenous Antibiotic, Na..."
0,Mappls vs Google Maps: 5 Features that make In...
4,Gemini Nano Banana trend: 5 AI prompts to crea...
